# Partie I : Collecte et traitement des données

In [ ]:
def download_data(company_name, params):
    params["apikey"] = api_key
    params["datatype"] = "csv"
    response = requests.get(url, params=params)

    if response.status_code == 200:
        filename = os.path.join("/home/onyxia/work/projet_gold_refuge/all_data_used", f"{company_name}_data.csv")
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Les données de {company_name} ont été sauvegardées dans {filename}.")
    else:
        print(f"Erreur pour {company_name}: {response.status_code}, {response.text}")

# Partie II : Partie descriptive

In [1]:
def convert_to_int(df, columns):
    df[columns] = df[columns].astype(int)
    return df

In [ ]:
def get_month_with_max_price(dataframe, price_column, date_column='year_month'):
    """
    Retourne le mois où une colonne de prix atteint sa valeur maximale.
    
    Args:
        dataframe (pd.DataFrame): Le DataFrame contenant les données.
        price_column (str): Le nom de la colonne de prix.
        date_column (str): Le nom de la colonne de date, par défaut 'year_month'.
    
    Returns:
        str: La valeur de la colonne de date correspondant au prix maximal.
    """
    max_price_row = dataframe[dataframe[price_column] == dataframe[price_column].max()]
    if not max_price_row.empty:
        return max_price_row[date_column].values[0]
    return None

# Partie III : Modélisation

In [ ]:
def analyze_company_gold(company_name):
    """
    Applique un modèle ARMA-GARCH univarié et effectue une régression pour estimer les coefficients de l'équation (3)

    Arguments :
    - company_name : Nom de l'entreprise.

    Retourne :
    - Un dictionnaire contenant le nom de l'entreprise et la somme c_GS et delta_GS.
    """
    # Charger les données globales
    global final_df

    # Vérifier que les colonnes nécessaires existent
    price_col = f"{company_name}_stock_price"
    if price_col not in final_df.columns:
        raise ValueError(f"La colonne {price_col} est absente du DataFrame.")

    # Préparer les données
    final_df = final_df[(final_df['gold_price'] > 0) & (final_df[price_col] > 0)]
    final_df = final_df.dropna()

    # Calculer les rendements logarithmiques
    final_df['gold_return'] = np.log(final_df['gold_price'] / final_df['gold_price'].shift(1)) * 100
    final_df[f'{company_name}_stock_return'] = np.log(final_df[price_col] / final_df[price_col].shift(1)) * 100
    final_df = final_df.dropna()

    # Ajuster les modèles GARCH univariés
    gold_model = arch_model(final_df['gold_return'], mean='Zero', vol='Garch', p=1, q=1)
    gold_fit = gold_model.fit(disp="off")

    company_model = arch_model(final_df[f'{company_name}_stock_return'], mean='Zero', vol='Garch', p=1, q=1)
    company_fit = company_model.fit(disp="off")

    # Extraire les résidus standardisés
    gold_std_resid = gold_fit.std_resid
    company_std_resid = company_fit.std_resid

    # Calculer les interactions (= produit des résidus) pour t-1
    final_df['interaction_t_minus_1'] = (gold_std_resid * company_std_resid).shift(1)
    final_df['interaction_dum_t_minus_1'] = (
        (gold_std_resid < 0).astype(int) * (company_std_resid < 0).astype(int) * gold_std_resid * company_std_resid
    ).shift(1)
    # 3. Calcul de la covariance conditionnelle
    final_df['covariance_conditionnelle'] = gold_std_resid * company_std_resid

    # Calcul de la covariance conditionnelle t-1
    final_df['h_t_minus_1'] = (gold_std_resid * company_std_resid).shift(1)

    # Ajouter la variable de crise
    final_df['dum_crise'] = (final_df['crisis'] == 1).astype(int)

    # Nettoyage des données
    final_df = final_df.replace([np.inf, -np.inf], np.nan).dropna()

    # Régression à partir de l'équation (3)
    X = final_df[['interaction_t_minus_1', 'h_t_minus_1', 'interaction_dum_t_minus_1', 'dum_crise']]
    X = add_constant(X)  # Ajouter une constante pour inclure c_GS
    y = final_df['covariance_conditionnelle']

    model_hGS = OLS(y, X)
    results_hGS = model_hGS.fit()

    # Extraire les coefficients
    c_GS = results_hGS.params['const']
    alpha_GS = results_hGS.params['interaction_t_minus_1']
    beta_GS = results_hGS.params['h_t_minus_1']
    d_GS = results_hGS.params['interaction_dum_t_minus_1']
    delta_GS = results_hGS.params['dum_crise']

    return {
        'company': company_name,
        'sum': c_GS + delta_GS
    }

In [ ]:
def analyze_company_bitcoin(company_name):
    """
    Applique un modèle ARMA-GARCH univarié et effectue une régression pour estimer c_GS et delta_GS pour une entreprise donnée.

    Arguments :
    - company_name : Nom de l'entreprise .

    Retourne :
    - Un dictionnaire contenant le nom de l'entreprise et la somme recherché.
    """
    # Charger les données globales
    global final_df

    # Vérifier que les colonnes nécessaires existent
    price_col = f"{company_name}_stock_price"
    if price_col not in final_df.columns:
        raise ValueError(f"La colonne {price_col} est absente du DataFrame.")

    # Préparer les données
    final_df = final_df[(final_df['bitcoin_price'] > 0) & (final_df[price_col] > 0)]
    final_df = final_df.dropna()

    # Calculer les rendements logarithmiques
    final_df['bitcoin_return'] = np.log(final_df['bitcoin_price'] / final_df['bitcoin_price'].shift(1)) * 100
    final_df[f'{company_name}_stock_return'] = np.log(final_df[price_col] / final_df[price_col].shift(1)) * 100
    final_df = final_df.dropna()

    # Ajuster les modèles GARCH univariés
    bitcoin_model = arch_model(final_df['bitcoin_return'], mean='Zero', vol='Garch', p=1, q=1)
    bitcoin_fit = bitcoin_model.fit(disp="off")

    company_model = arch_model(final_df[f'{company_name}_stock_return'], mean='Zero', vol='Garch', p=1, q=1)
    company_fit = company_model.fit(disp="off")

    # Extraire les résidus standardisés
    bitcoin_std_resid = bitcoin_fit.std_resid
    company_std_resid = company_fit.std_resid

    # Calculer les interactions (= produit des résidus) pour t-1
    final_df['interaction_t_minus_1'] = (bitcoin_std_resid * company_std_resid).shift(1)
    final_df['interaction_dum_t_minus_1'] = (
        (bitcoin_std_resid < 0).astype(int) * (company_std_resid < 0).astype(int) * bitcoin_std_resid * company_std_resid
    ).shift(1)

    # Calcul de la covariance conditionnelle
    final_df['covariance_conditionnelle'] = bitcoin_std_resid * company_std_resid

    # Calcul de la covariance conditionnelle t-1
    final_df['h_t_minus_1'] = (bitcoin_std_resid * company_std_resid).shift(1)

    # Ajouter la variable de crise
    final_df['dum_crise'] = (final_df['crisis'] == 1).astype(int)

    # Nettoyage des données
    final_df = final_df.replace([np.inf, -np.inf], np.nan).dropna()

    # Régression à partir de l'équation (3)
    X = final_df[['interaction_t_minus_1', 'h_t_minus_1', 'interaction_dum_t_minus_1', 'dum_crise']]
    X = add_constant(X)  # Ajouter une constante pour inclure c_GS
    y = final_df['covariance_conditionnelle']

    model_hGS = OLS(y, X)
    results_hGS = model_hGS.fit()

    # Extraire les coefficients
    c_GS = results_hGS.params['const']
    delta_GS = results_hGS.params['dum_crise']

    return {
        'company': company_name,
        'sum': c_GS + delta_GS
    }

In [ ]:
def analyze_company_ethereum(company_name):
    """
    Applique un modèle ARMA-GARCH univarié et effectue une régression pour estimer c_GS et delta_GS pour une entreprise donnée.

    Arguments :
    - company_name : Nom de l'entreprise.

    Retourne :
    - Un dictionnaire contenant c_GS, delta_GS et leur somme.
    """
    # Charger les données globales
    global final_df

    # Vérifier que les colonnes nécessaires existent
    price_col = f"{company_name}_stock_price"
    if price_col not in final_df.columns:
        raise ValueError(f"La colonne {price_col} est absente du DataFrame.")

    # Préparer les données
    final_df = final_df[(final_df['ethereum_price'] > 0) & (final_df[price_col] > 0)]
    final_df = final_df.dropna()

    # Calculer les rendements logarithmiques
    final_df['ethereum_return'] = np.log(final_df['ethereum_price'] / final_df['ethereum_price'].shift(1)) * 100
    final_df[f'{company_name}_stock_return'] = np.log(final_df[price_col] / final_df[price_col].shift(1)) * 100
    final_df = final_df.dropna()

    # Ajuster les modèles GARCH univariés
    ethereum_model = arch_model(final_df['ethereum_return'], mean='Zero', vol='Garch', p=1, q=1)
    ethereum_fit = ethereum_model.fit(disp="off")

    company_model = arch_model(final_df[f'{company_name}_stock_return'], mean='Zero', vol='Garch', p=1, q=1)
    company_fit = company_model.fit(disp="off")

    # Extraire les résidus standardisés
    ethereum_std_resid = ethereum_fit.std_resid
    company_std_resid = company_fit.std_resid

    # Calculer les interactions (= produit des résidus) pour t-1
    final_df['interaction_t_minus_1'] = (ethereum_std_resid * company_std_resid).shift(1)
    final_df['interaction_dum_t_minus_1'] = (
        (ethereum_std_resid < 0).astype(int) * (company_std_resid < 0).astype(int) * ethereum_std_resid * company_std_resid
    ).shift(1)

    # Calcul de la covariance conditionnelle
    final_df['covariance_conditionnelle'] = ethereum_std_resid * company_std_resid

    # Calcul de la covariance conditionnelle t-1
    final_df['h_t_minus_1'] = (ethereum_std_resid * company_std_resid).shift(1)

    # Ajouter la variable de crise
    final_df['dum_crise'] = (final_df['crisis'] == 1).astype(int)

    # Nettoyage des données
    final_df = final_df.replace([np.inf, -np.inf], np.nan).dropna()

    # Régression à partir de l'équation (3)
    X = final_df[['interaction_t_minus_1', 'h_t_minus_1', 'interaction_dum_t_minus_1', 'dum_crise']]
    X = add_constant(X)  # Ajouter une constante pour inclure c_GS
    y = final_df['covariance_conditionnelle']

    model_hGS = OLS(y, X)
    results_hGS = model_hGS.fit()

    # Extraire les coefficients
    c_GS = results_hGS.params['const']
    delta_GS = results_hGS.params['dum_crise']

    return {
        'company': company_name,
        'sum': c_GS + delta_GS
    }